<h1><b>AIのべりすと非公式トークン化ツール</b></h1>
    <h3>「AIのべりすと」で使用されているトークナイザー（SentencePiece）で文章をトークンごとに分割します。</h3>
<font size=3>
<p>説明：</p>
    <ul>
        <li>入力した文章のトークン数をカウントします。</li>
        <li>トークンに無い文字を抽出、強調します。</li>
    </ul>
<p>注意：</p>
    <ul>
        <li>非公式ツールです。実際の仕様とは若干異なる可能性があります。</li>
        <li>入力文はGoogleのクラウドサーバーで処理されます。</li>
    </ul>
</font>

リンク： [AIのべりすと](https://ai-novel.com)

---

In [ ]:
# Author: ドングリネコ (acorncat)
# Version: 0.2

#@title # ◆【初期設定】 { vertical-output: true, display-mode: "form" }
#@markdown ### ←最初に実行ボタンを押してください（数十秒かかります） { vertical-output: true, display-mode: "form" }
#@markdown <br>完了したら下へ進んでください。
if "sp" not in locals():
    print("しばらくお待ち下さい。")

    !wget -O spiece_old.model https://raw.githubusercontent.com/acorncat/unofficial-tools/main/models/spiece.model &>/dev/null
    !wget -O spiece_new.model https://huggingface.co/naclbit/trin_tokenizer_v3/resolve/main/spiece.model &>/dev/null

    # SentencePieceをインポート
    !pip install sentencepiece &>/dev/null
    import sentencepiece as spm
    sp = spm.SentencePieceProcessor(model_file = "spiece_old.model")
    # 「◇トークン検索」用にトークンリストを作成
    tokens_l_old = [sp.IdToPiece(i) for i in range(sp.GetPieceSize())]

    # やみ/スパとり用
    sp = spm.SentencePieceProcessor(model_file = "spiece_new.model")
    tokens_l_new = [sp.IdToPiece(i) for i in range(sp.GetPieceSize())]

    import re
    from IPython.display import HTML
    display(HTML("<h1>完了！</h1>"))

else: print("〈エラー〉既に実行済みです。")

def eap_sgpass(sentence, model_name):
    # サロゲートペアや絵文字対策（参考：https://stackoverflow.com/questions/38147259/how-can-i-convert-surrogate-pairs-to-normal-string-in-python）
    sentence = sentence.encode("utf-16", "surrogatepass").decode("utf-16")

    if "やみ" in model_name:
        sp = spm.SentencePieceProcessor(model_file = "spiece_new.model")
    else:
        sp = spm.SentencePieceProcessor(model_file = "spiece_old.model")

    pieces = sp.EncodeAsPieces(sentence)
    return pieces

---

<h1>項目の説明</h1>
<p>
<font size=4>
    <ul>
        <li>sentence: 文章の入力欄です。ここに文章を入力して、実行ボタンを押してください。複数行入力したい場合はペーストしてください。※数万文字以上ペーストするとブラウザがフリーズする可能性があります。
        </li>
        <li>model_name: 使用するモデルを選択してください。「やみおとめ/スーパーとりんさま」、「とりんさま/でりだ」の2つから選べます。
        </li>
        <li>space: スペース・改行をこれに置換します。</li>
        <li>cut_mark: トークンを区切る文字です。</li>
        <li>check_unk: トークンに無い文字を強調します。AIには&lt;unk&gt;というトークンして認識されます。</li>
        <li>fontsize: 結果の文字サイズを変更します。※変更後、再度実行しないと反映されません。</li>
    </ul>
</font>
</p>

In [ ]:
#@title # ◆【トークン数カウント】 { display-mode: "form" }

sentence = "" #@param {type:"string"}
model_name = "\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E" # @param ["\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E", "\u3068\u308A\u3093\u3055\u307E/\u3067\u308A\u3060"]
space = "\u6539\u884C" #@param ["改行", "改行（\\nを表示）", "▁", "四角つき全角スペース", "半角スペース"]
cut_mark = "\uFF5C" #@param ["｜", "/", ",", "無し"]
check_unk = "\u8D64\u6587\u5B57\u306B\u3059\u308B" #@param ["赤文字にする", "四角で囲む"]
fontsize = 1.2 #@param {type:"slider", min:1, max:3, step:0.1}

result = ""

# 未知語の強調CSS
if check_unk == "赤文字にする":
    unk_style = "color: red;"
elif check_unk == "四角で囲む":
    unk_style = "border: 1px solid;"
# 結果全体のCSS（フォントサイズ）
css = ("<style>.result{font-size:" + str(fontsize) + "rem;}"
     + ".unk{" + unk_style + "}.gray{color: rgb(155, 155, 155);}</style>")

if "sp" in locals():
    # 入力文をトークン化
    pieces = eap_sgpass(sentence, model_name)
    if "やみ" in model_name:
        sp = spm.SentencePieceProcessor(model_file = "spiece_new.model")
    else:
        sp = spm.SentencePieceProcessor(model_file = "spiece_old.model")

    # トークン列をIDに変換し、未知語<unk>（ID 1）かどうか確認
    michi = ""
    for index, p in enumerate(pieces):
        if sp.PieceToId(p) == 1:
            michi += p
            # 未知語を置換＜全角＞
            pieces[index] = "＜span class=\"unk\"＞" + pieces[index] + "＜/span＞"

    # 脱トークン化
    result += ("【合計 " + str(len(pieces)) + "トークン】＜br＞"
             + "｜".join(pieces))

    result = result.replace("&", "&amp;")
    result = result.replace("<", "&lt;").replace(">", "&gt;")
    result = result.replace("＜", "<").replace("＞", ">")
    result = result.replace("!", "！").replace("?", "？")

    # スペースの置換
    if space != "▁":
        if space == "改行（\\nを表示）":
            result = re.sub("▁(｜)?", "<span class=\"gray\">\\\\n\\1</span><br>", result)
        elif space == "改行":
            result = re.sub("▁(｜)?", "\\1<br>", result)
        else:
            space = space.replace("半角スペース", " ")
            space = space.replace("四角つき全角スペース", "<span style=\"border: 1px solid;\">　</span>")
            result = result.replace("▁", space)
    # 区切り文字の置換
    if cut_mark == "無し":
        cut_mark = ""
    else:
        cut_mark = "<span class=\"gray\">" + cut_mark + "</span>"
        cut_mark = cut_mark.replace(",", ", ")
    result = result.replace("｜", cut_mark)

    # 未知語があれば重複を削除して表示
    if len(michi) >= 1:
        michi_remove = dict.fromkeys(list(michi))
        result = ("【トークンに無い文字が " + str(len(michi_remove)) + " 個見つかりました】 <br>"
                 + ",".join(michi_remove) + "<br>" + result)

    display_result = css + "<p class=\"result\">" + result + "</p>"
    # 合計トークン数と結果を表示
    display(HTML(display_result))


else:
    print("〈エラー〉先に【初期設定】を実行してください。")

In [ ]:
#@title # ◇【メモ欄】 { display-mode: "form" }
#@markdown 実行不要です。メモ欄が下に表示されていないときのみ実行してください。
def memo():
    display(HTML("""
    <style>
        input[type=range] {
            -webkit-appearance: none;
            height: 12px;
            width: 50%;
        }
        input[type=range]::-webkit-slider-thumb {
            width: 30px;
            height: 30px;
        }
        input[type=range]::-moz-range-thumb {
            width: 30px;
            height: 30px;
        }
        input[type=range]::-ms-thumb {
            width: 30px;
            height: 30px;
        }
        textarea {
            font-size: 1.2rem;
            width: 38rem;
            height: 25rem;
            border: 1px solid;
        }
        button {
            font-size: 1.3rem;
            width: 10rem;
            height: 3rem;
            border: 1px solid;
            border-radius: 5px;
        }
        .light {
            background-color: rgb(247, 247, 247);
            color: black;
        }
        .dark {
            background-color: rgb(30, 30, 30);
            color: rgb(212, 212, 212);
        }
    </style>

    <script>
        const input_fontsize = document.querySelector("input");
        const cur_fontsize = document.getElementById("cur_fontsize");
        const memo = document.querySelector("textarea");
        const light_button = document.getElementById("light_button");
        const dark_button = document.getElementById("dark_button");

        input_fontsize.addEventListener("input", updateValue);
        light_button.addEventListener("click", click_light);
        dark_button.addEventListener("click", click_dark);

        function updateValue(e) {
            font_size = e.target.value;
            cur_fontsize.textContent = font_size;
            memo.style.fontSize = font_size + "rem";
        }
        function click_light() {
            memo.className = "light";
            light_button.className = "light";
            dark_button.className = "light";
        }
        function click_dark() {
            memo.className = "dark";
            light_button.className = "dark";
            dark_button.className = "dark";
        }
    </script>
    <input type="range" min="1" max="3" step="0.1" value="1.2"/>
    <lavel style="font-size:1.7rem">メモの文字サイズ:<span id="cur_fontsize">1.2</span></font></lavel>
    <p>
    <button class="light" type="button" id="light_button">明るく</button>
    <button class="light" type="button" id="dark_button">暗く</button>
    </p>
    <textarea class="light" placeholder="ご自由にお使いください。ただのメモ欄なので、送信はできません。"></textarea>
    """))
try:
    memo()
except NameError:
    from IPython.display import HTML
    memo()

In [ ]:
#@title ## ◆トークン検索 { display-mode: "form" }
#@markdown 入力した文字列が含まれているトークンを検索します。
search_word = "" #@param {type:"string"}
model_name = "\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E" # @param ["\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E", "\u3068\u308A\u3093\u3055\u307E/\u3067\u308A\u3060"]

def search_tokens_list(search_word, model_name):
    if len(search_word) <= 0:
        print("1文字以上入力してください。")
        return
    search_word = search_word.replace(" ", "▁")

    if "やみ" in model_name:
        result_search = [t for t in tokens_l_new if search_word in t]
    else:
        result_search = [t for t in tokens_l_old if search_word in t]

    if len(result_search) >= 1:
        print("【", len(result_search), "個のトークンが見つかりました】")
        print("\n".join(result_search))
    else:
        print("見つかりませんでした。")

if "sp" in locals():
    search_tokens_list(search_word, model_name)
else:
    print("〈エラー〉先に【初期設定】を実行してください。")

In [ ]:
#@title ## ◆トークンIDに変換 { display-mode: "form" }
#@markdown 入力した文章をトークンに分割し、トークンIDに変換します。
sentence = "" #@param {type:"string"}
model_name = "\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E" # @param ["\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E", "\u3068\u308A\u3093\u3055\u307E/\u3067\u308A\u3060"]


def check_ids(sentence):
    if len(sentence) <= 0:
        print("1文字以上入力してください。")
        return

    if "やみ" in model_name:
        sp = spm.SentencePieceProcessor(model_file = "spiece_new.model")
    else:
        sp = spm.SentencePieceProcessor(model_file = "spiece_old.model")

    pieces = eap_sgpass(sentence, model_name)
    for index, p in enumerate(pieces):
        pieces[index] = (p + " (" +  str(sp.PieceToId(p)) + ")")
    result = "\n".join(pieces)
    print(result)

try:
    check_ids(sentence)
except NameError:
    print("〈エラー〉先に【初期設定】を実行してください。")

In [ ]:
#@title ## ◆トークン数カウント（シンプル版） { display-mode: "form" }
#@markdown 結果は1行で出力されます。
sentence = "" #@param {type:"string"}
model_name = "\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E" # @param ["\u3084\u307F\u304A\u3068\u3081/\u30B9\u30FC\u30D1\u30FC\u3068\u308A\u3093\u3055\u307E", "\u3068\u308A\u3093\u3055\u307E/\u3067\u308A\u3060"]

sentence = sentence.encode("utf-16", "surrogatepass").decode("utf-16")

if "sp" in locals():

    if "やみ" in model_name:
        sp = spm.SentencePieceProcessor(model_file = "spiece_new.model")
    else:
        sp = spm.SentencePieceProcessor(model_file = "spiece_old.model")

    # トークン化
    pieces = eap_sgpass(sentence, model_name)

    # 未知語の確認
    michi = ""
    for index, t in enumerate(pieces):
      if sp.PieceToId(t) == 1:
        michi += pieces[index]
        pieces[index] = "<unk>"

    if len(michi) != 0:
        # リスト化した後、重複を削除
        michi_remove = dict.fromkeys(list(michi))
        print("未知語", len(michi_remove))
        print(",".join(michi_remove))

    print("トークン数", len(pieces))
    print("/".join(pieces))
else:
    print("〈エラー〉先に【初期設定】を実行してください。")